# Integrazione di Semantic Kernel con il server OpenBnB MCP

Questo notebook mostra come utilizzare Semantic Kernel con il server OpenBnB MCP reale per cercare alloggi Airbnb autentici utilizzando MCPStdioPlugin. Per l'accesso LLM, utilizza Azure AI Foundry. Per configurare le variabili d'ambiente, puoi seguire la [Lezione di configurazione](/00-course-setup/README.md)


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Creazione della connessione al plugin MCP

Ci collegheremo al [server MCP di OpenBnB](https://github.com/openbnb-org/mcp-server-airbnb) utilizzando MCPStdioPlugin. Questo server offre funzionalità di ricerca Airbnb tramite il pacchetto @openbnb/mcp-server-airbnb.


## Creazione del Client

In questo esempio, utilizzeremo Azure AI Foundry per accedere al nostro LLM. Assicurati che le variabili d'ambiente siano configurate correttamente.


## Configurazione dell'Ambiente

Configura le impostazioni di Azure OpenAI. Assicurati di avere le seguenti variabili di ambiente impostate:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Comprendere l'integrazione OpenBnB MCP

Questo notebook si connette al **vero server OpenBnB MCP**, che fornisce funzionalità di ricerca reali per Airbnb.

### Come funziona:

1. **MCPStdioPlugin**: Utilizza la comunicazione tramite input/output standard con il server MCP
2. **Pacchetto NPM reale**: Scarica ed esegue `@openbnb/mcp-server-airbnb` tramite npx
3. **Dati in tempo reale**: Restituisce dati reali sulle proprietà Airbnb dalle loro API
4. **Scoperta delle funzioni**: L'agente scopre automaticamente le funzioni disponibili dal server MCP

### Funzioni disponibili:

Il server OpenBnB MCP generalmente fornisce:
- **search_listings** - Cerca proprietà Airbnb per località e criteri
- **get_listing_details** - Ottieni informazioni dettagliate su proprietà specifiche
- **check_availability** - Verifica la disponibilità per date specifiche
- **get_reviews** - Recupera recensioni per le proprietà
- **get_host_info** - Ottieni informazioni sui proprietari delle proprietà

### Prerequisiti:

- **Node.js** installato sul tuo sistema
- **Connessione Internet** per scaricare il pacchetto del server MCP
- **NPX** disponibile (incluso con Node.js)

### Testare la connessione:

Puoi testare manualmente il server MCP eseguendo:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Questo scaricherà e avvierà il server OpenBnB MCP, al quale Semantic Kernel si connetterà per ottenere dati reali da Airbnb.


## Esecuzione dell'Agent con il server OpenBnB MCP

Ora eseguiremo l'AI Agent che si connette al server OpenBnB MCP per cercare alloggi reali su Airbnb a Stoccolma per 2 adulti e 1 bambino. Sentiti libero di modificare la lista `user_inputs` per cambiare i criteri di ricerca.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Sommario  
Congratulazioni! Hai costruito con successo un agente AI che si integra con la ricerca di alloggi reali utilizzando il Model Context Protocol (MCP):

Tecnologie Utilizzate:  
Semantic Kernel - Per creare agenti intelligenti con Azure OpenAI  
Azure AI Foundry - Per capacità LLM e completamento delle chat  
MCP (Model Context Protocol) - Per l'integrazione standardizzata degli strumenti  
OpenBnB MCP Server - Per la funzionalità reale di ricerca su Airbnb  
Node.js/NPX - Per eseguire il server MCP esterno  

Cosa Hai Imparato:  
Integrazione MCP: Collegare agenti Semantic Kernel a server MCP esterni  
Accesso ai Dati in Tempo Reale: Ricerca di proprietà Airbnb reali tramite API live  
Comunicazione del Protocollo: Utilizzo della comunicazione stdio tra agente e server MCP  
Scoperta delle Funzioni: Scoprire automaticamente le funzioni disponibili dai server MCP  
Risposte in Streaming: Catturare e registrare le chiamate alle funzioni in tempo reale  
Rendering HTML: Formattare le risposte dell'agente con tabelle stilizzate e display interattivi  

Prossimi Passi:  
Integrare server MCP aggiuntivi (meteo, voli, ristoranti)  
Costruire un sistema multi-agente combinando MCP e protocolli A2A  
Creare server MCP personalizzati per le tue fonti di dati  
Implementare memoria conversazionale persistente tra sessioni  
Distribuire l'agente su Azure Functions con orchestrazione del server MCP  
Aggiungere autenticazione utente e funzionalità di prenotazione  

Vantaggi Chiave dell'Architettura MCP:  
Standardizzazione: Protocollo universale per collegare agenti AI a strumenti esterni  
Dati in Tempo Reale: Accesso a informazioni aggiornate e live da vari servizi  
Estensibilità: Integrazione facile di nuove fonti di dati e strumenti  
Interoperabilità: Funziona su diversi framework AI e piattaforme di agenti  
Separazione delle Responsabilità: Distinzione chiara tra logica AI e accesso ai dati esterni  



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
